# Google Places API - Determine Operational Status

Author: Trang Van

**Note:** If using this notebook for individual searching, run PART 0 and 1 before skipping to PART 3.

In [ ]:
%pip install dms2dec

# Note: Ended up not using for the rest of the notebook. Useful for Google Places' Nearby Search (see resources for how-to)

In [ ]:
## SAMPLE USAGE OF dms2dec LIBRARY
from dms2dec.dms_convert import dms2dec
# Convert Coordinates from (Degree Minute Seconds) to (Decimal)
lat = '''37° 44' 11.4" N'''
lon = "122° 11' 40.6\" W"
str(dms2dec(lat)) + "," + str(dms2dec(lon))

### **0. RUN FOLLOWING CELL BEFORE MOVING ON TO ANY OTHER PART.**

In [1]:
import requests
import json
import time
import pandas as pd

## 1. Making the GooglePlaces Class 

Adapted from MAJID ALIZADEH: https://python.gotrained.com/google-places-api-extracting-location-data-reviews/

The following changes will extract Basic Data Fields like `name`, `business_status`, etc. 

To use, create a `GooglePlaces()` object and pass in your API key. Specify what fields you want to extract (Basic Data is free). Search by input first and then use the returned Place ID to get a Place's details. See part 2 for example uses.


In [12]:
# REMOVE BEFORE UPLOADING TO PUBLIC REPO
api_key = ""

In [7]:
# Modified from: https://python.gotrained.com/google-places-api-extracting-location-data-reviews/
class GooglePlaces(object):
    def __init__(self, api_key):
        super(GooglePlaces, self).__init__()
        self.api_key = api_key
 
    def search_by_input(self, location, fields, study_area='Oakland'):
        """
        Returns KEY-VALUE Pair(s) of place_id's from Google Places' search via TEXT input.

        INPUTS:
        location: STRING input of location (Name, Address, etc.) 
        fields: LIST of Google Places Data Fields (see Places API Doc)
        study_area: STRING to narrow down searches with multiple results
        
        OUTPUT:
        KEY-VALUE Pair(s)
        """
        place_id = []
        query_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
        params = {
            'input': location,
            'inputtype': "textquery",
            'fields': fields,
            'key': self.api_key
        }
        res = requests.get(query_url, params = params)
        results =  json.loads(res.content)
        place_id.extend(results['candidates'])
        #print(place_id)
        if len(place_id) == 0:
            # Case 1: No place_id in Places API
            return {'place_id': ""}
        elif len(place_id) > 1:
            # Case 2: Multiple locations with input, filter for study
            for item in place_id:
                details = self.get_place_details(item['place_id'],fields)
                address = details['result']['formatted_address']
                if study_area not in address.upper():
                    place_id.remove(item)
            print("Edited List:", place_id)
        # Case 3: One place resulted from search, return the only one
        return place_id[0]
 
    def get_place_details(self, place_id, fields):
        """
        Returns JSON format of Place's details (with user-specified fields) using unique place_id.
        
        INPUTS:
        place_id: STRING
        fields: LIST
        """
        query_url = "https://maps.googleapis.com/maps/api/place/details/json"
        params = {
            'placeid': place_id,
            'fields': ",".join(fields),
            'key': self.api_key
        }
        res = requests.get(query_url, params=params)
        place_details =  json.loads(res.content)
        return place_details
    
    def get_bus_status(self, place_id):
        """
        Returns status of business based on place ID. Converts Google's results to standard for project.
        Primarily used for larger scale cases (eg. filling in Dataframe column)
        
        USE CASE: 
        refined_orbis_ppp['Company name Latin alphabet'].apply(lambda x: api.get_bus_status(api.search_by_input(location=x, fields=fields, study_area=study_area)['place_id']))
        
        INPUT:
        place_id: STRING 
        """
        if place_id == "":
            # Edge Case: No available place_id in API
            return ""
        
        query_url = "https://maps.googleapis.com/maps/api/place/details/json"
        params = {
            'placeid': place_id,
            'fields': 'business_status',
            'key': self.api_key
        }
        res = requests.get(query_url, params=params)
        results =  json.loads(res.content)
        
        business_status = results['result']['business_status']
        if business_status == 'OPERATIONAL':
            return 'Active'
        elif business_status == 'CLOSED_PERMANENTLY':
            return 'Closed'
        elif business_status == 'CLOSED_TEMPORARILY':
            return 'Temporarily Closed'
        else:
            return 'Unknown'
        
        # What about: 'Expired license', 'Not in license database'
        return ""
    
    def display_fields(self, details, fields):
        """
        Prints to STDOUT the value of each fields for a place.
        
        INPUT:
        
        details: JSON with key-value pairs for each field
        fields: LIST
        """
        print("===================PLACE===================") 
        for f in fields:
            try:
                print(f.replace("_", " ").title() + ": " + str(details['result'][f]))
            except KeyError:
                print(f.title() + ": "+"N/A")

## 2a. Simple Example - One Location

Using one location name (eg. DIGICOM in Oakland) as sample input. Testing all four functions in class.

In [153]:
# User-specified fields (see full list here: https://developers.google.com/maps/documentation/places/web-service/place-data-fields)
fields = ['place_id', 'name', 'formatted_address', 'business_status','permanently_closed','type', 'url']

In [157]:
# Construct GooglePlaces object and search for business
api = GooglePlaces(api_key)
place = api.search_by_input(location='DIGICOM', fields=fields)

In [158]:
# Get details of the place and then print it out.
details = api.get_place_details(place['place_id'], fields)
api.display_fields(details, fields)

===================PLACE===================
Place Id: ChIJRxkWkd94hYARvUk7nHmfIHQ
Name: Digicom Electronics
Formatted Address: 7799 Pardee Ln, Oakland, CA 94621, USA
Business Status: OPERATIONAL
Permanently_Closed: N/A
Type: N/A
Url: https://maps.google.com/?cid=8367863452315371965


In [160]:
# Test Usage for `get_bus_status` (Expecting: `Active`)
api.get_bus_status('ChIJRxkWkd94hYARvUk7nHmfIHQ')

'Active'

## 2b. Determine Operational Status - Fill in whole column.

Now, we'll read in the Orbis and PPP merged file which contains businesses across all study areas. We will specify one area (Redwood City) and test on 100 rows.

**OPERATIONAL STATUS CRITERIA:**<br>
If the business has an active license based on the business license data from your city list it as `Active`

If the business has an expired license, list as `Expired license`

If the business is closed, list as `Closed`

If the business does not appear in the business license database at all note this as `Not in license database`

In [43]:
# Read in Orbis and PPP loan data.
orbis_ppp_df = pd.read_csv('ppp-loan-data/out/orbis-ppp-master.csv', low_memory=False)
orbis_ppp_df.head()

,Company name Latin alphabet,Inactive,Quoted,Branch,OwnData,Woco,Country ISO code,"NACE Rev. 2, core code (4 digits)",Consolidation code,Last avail. year,...,Study Area,Operational status,CurrentApprovalAmount,YearApproved,Race,Ethnicity,Minority,NAICS_4,IndustrySubsector,Company Name - Clean
0,DICKS WHOLESALE CARPET WAREHOUSE INC,No,No,No,No,No,US,4778.0,LF,2019.0,...,OAKLAND,NaN,169656.0,2021.0,Unanswered,Unknown/NotStated,Unanswered,4422.0,Home Furnishings Stores,DICKS WHOLESALE CARPET WAREHOUSE
1,DIGICOM,No,No,No,No,No,US,4778.0,LF,2019.0,...,OAKLAND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DIGICOM
2,BAY AREA CONTRACT CARPETS INC,No,No,No,No,No,US,4778.0,LF,2019.0,...,OAKLAND,NaN,74219.0,2020.0,Unanswered,Unknown/NotStated,Unanswered,4422.0,Home Furnishings Stores,BAY AREA CONTRACT CARPETS
3,FRIANT & ASSOCIATES INC,No,No,No,No,No,US,4779.0,LF,2019.0,...,OAKLAND,NaN,2842000.0,2020.0,Unanswered,Unknown/NotStated,Unanswered,4421.0,Furniture Stores,FRIANT & ASSOCIATES
4,FRIANT & ASSOCIATES INC,No,No,No,No,No,US,4779.0,LF,2019.0,...,OAKLAND,NaN,2000000.0,2021.0,Unanswered,Unknown/NotStated,Unanswered,4421.0,Furniture Stores,FRIANT & ASSOCIATES


In [51]:
# Specify study area (look at `Study Area` column in dataframe)
study_area = 'REDWOOD CITY'

In [133]:
refined_orbis_ppp = orbis_ppp_df[orbis_ppp_df['Study Area'] == study_area][:100]
refined_orbis_ppp.head()

,Company name Latin alphabet,Inactive,Quoted,Branch,OwnData,Woco,Country ISO code,"NACE Rev. 2, core code (4 digits)",Consolidation code,Last avail. year,...,Study Area,Operational status,CurrentApprovalAmount,YearApproved,Race,Ethnicity,Minority,NAICS_4,IndustrySubsector,Company Name - Clean
7282,SKY ZONE,No,No,No,No,No,US,4778.0,LF,2019.0,...,REDWOOD CITY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SKY ZONE
7283,"LAUNCHPAD, INC.",No,No,No,No,No,US,4778.0,LF,2020.0,...,REDWOOD CITY,NaN,642400.00,2020.0,Unanswered,Unknown/NotStated,Unanswered,4532.0,"Office Supplies, Stationery, and Gift Stores",LAUNCHPAD
7284,COAST LIGHTING INC,No,No,No,No,No,US,4759.0,LF,2019.0,...,REDWOOD CITY,NaN,199540.00,2020.0,White,Unknown/NotStated,No,4422.0,Home Furnishings Stores,COAST LIGHTING
7285,REDWOOD TRADING POST,No,No,No,No,No,US,4764.0,LF,2019.0,...,REDWOOD CITY,NaN,85077.42,2020.0,Unanswered,Unknown/NotStated,Unanswered,4539.0,Other Miscellaneous Store Retailers,REDWOOD TRADING POST
7286,KOLKKA FURNITURE,No,No,No,No,No,US,4759.0,LF,2019.0,...,REDWOOD CITY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KOLKKA FURNITURE


In [128]:
api = GooglePlaces(api_key)

In [134]:
refined_orbis_ppp['Operational status'] = refined_orbis_ppp['Company name Latin alphabet'].apply(lambda x: api.get_bus_status(api.search_by_input(location=x, fields=fields, study_area=study_area)['place_id']))

Edited List: [{'place_id': 'ChIJ2QHakAijj4AR5drwpOWac9M'}]
Edited List: [{'place_id': 'ChIJrclrJ1Oij4ARGae3NgmRTvo'}]
Edited List: [{'place_id': 'ChIJkc3540yij4ARYEy5lkWmNwA'}]
Edited List: [{'place_id': 'ChIJG2IhMK2jj4ARvGp1gUZxTdI'}]
Edited List: [{'place_id': 'ChIJz6vJUVWij4ARV3vvg6RHXAo'}]
Edited List: [{'place_id': 'ChIJd2ozDayjj4ARRfACIFI2LuY'}]
Edited List: [{'place_id': 'ChIJ704BMSefj4ARMqCeKg7UNrQ'}]
Edited List: [{'place_id': 'ChIJ9Ssy7e2gj4ARwaCZdcZT72o'}]
Edited List: [{'place_id': 'ChIJWenjBAyjj4ARDIQQprzZOBE'}]


In [136]:
refined_orbis_ppp.head(10)

,Company name Latin alphabet,Inactive,Quoted,Branch,OwnData,Woco,Country ISO code,"NACE Rev. 2, core code (4 digits)",Consolidation code,Last avail. year,...,Study Area,Operational status,CurrentApprovalAmount,YearApproved,Race,Ethnicity,Minority,NAICS_4,IndustrySubsector,Company Name - Clean
7282,SKY ZONE,No,No,No,No,No,US,4778.0,LF,2019.0,...,REDWOOD CITY,Active,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SKY ZONE
7283,"LAUNCHPAD, INC.",No,No,No,No,No,US,4778.0,LF,2020.0,...,REDWOOD CITY,Active,642400.00,2020.0,Unanswered,Unknown/NotStated,Unanswered,4532.0,"Office Supplies, Stationery, and Gift Stores",LAUNCHPAD
7284,COAST LIGHTING INC,No,No,No,No,No,US,4759.0,LF,2019.0,...,REDWOOD CITY,Active,199540.00,2020.0,White,Unknown/NotStated,No,4422.0,Home Furnishings Stores,COAST LIGHTING
7285,REDWOOD TRADING POST,No,No,No,No,No,US,4764.0,LF,2019.0,...,REDWOOD CITY,Active,85077.42,2020.0,Unanswered,Unknown/NotStated,Unanswered,4539.0,Other Miscellaneous Store Retailers,REDWOOD TRADING POST
7286,KOLKKA FURNITURE,No,No,No,No,No,US,4759.0,LF,2019.0,...,REDWOOD CITY,Active,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KOLKKA FURNITURE
7287,CITY PUB,No,No,No,No,No,US,5630.0,LF,2019.0,...,REDWOOD CITY,Active,135449.00,2020.0,White,Not Hispanic or Latino,No,7225.0,Restaurants and Other Eating Places,CITY PUB
7288,CITY PUB,No,No,No,No,No,US,5630.0,LF,2019.0,...,REDWOOD CITY,Active,150000.00,2021.0,White,Not Hispanic or Latino,No,7225.0,Restaurants and Other Eating Places,CITY PUB
7289,SIGONA'S SIGNATURE COLLECTIONS,No,No,No,No,No,US,4778.0,LF,2019.0,...,REDWOOD CITY,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIGONA'S SIGNATURE COLLECTIONS
7290,GORIDE BICYCLES,No,No,No,No,No,US,4764.0,LF,2018.0,...,REDWOOD CITY,Active,70830.00,2020.0,Unanswered,Unknown/NotStated,Unanswered,4511.0,"Sporting Goods, Hobby, and Musical Instrument ...",GORIDE BICYCLES
7291,BEN FRANKS,No,No,No,No,No,US,5610.0,LF,2019.0,...,REDWOOD CITY,Active,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BEN FRANKS


### Notes: Multiple Places in One Search

In [113]:
places = [[{'place_id': 'ChIJTzDM6TS7j4AR8j68K7Ov0Dk'}, {'place_id': 'ChIJ2QHakAijj4AR5drwpOWac9M'}], [{'place_id': 'ChIJN_ReNjN2j4AR11CThpjoosc'}, {'place_id': 'ChIJrclrJ1Oij4ARGae3NgmRTvo'}], [{'place_id': 'ChIJkc3540yij4ARYEy5lkWmNwA'}, {'place_id': 'ChIJ3YOe5-h-hYARn07WH6Af6J0'}]
,[{'place_id': 'ChIJG2IhMK2jj4ARvGp1gUZxTdI'}, {'place_id': 'ChIJBVY2Bxh-j4ARa2zO8Jd6H2A'}],[{'place_id': 'ChIJz6vJUVWij4ARV3vvg6RHXAo'}, {'place_id': 'ChIJA8kJqr2Fj4ARdYDoP4fOOqc'}]]

for place in places:
    for p in place: 
        details = api.get_place_details(p['place_id'], fields)
        try:
            place_id = details['result']['place_id']
        except KeyError:
            place_id = ""
        try:
            name = details['result']['name']
        except KeyError:
            name = ""
        try:
            address = details['result']['formatted_address']
        except KeyError:
            address = ""
        try:
            business_status = details['result']['business_status']
        except KeyError:
            business_status = ""
        try:
            permanently_closed = details['result']['permanently_closed']
        except KeyError:
            permanently_closed = "N/A"
        try:
            url = details['result']['url']
        except KeyError:
            url = ""

        print("===================PLACE===================")
        print("Place ID:", place_id)
        print("Name:", name)
        print("Address:", address)
        print("Business Status:", business_status)
        print("Permanently Closed:", permanently_closed)
        print("URL:", url)

===================PLACE===================
Place ID: ChIJTzDM6TS7j4AR8j68K7Ov0Dk
Name: Sigona's Farmers Market
Address: 399 Stanford Shopping Center, Palo Alto, CA 94304, USA
Business Status: OPERATIONAL
Permanently Closed: N/A
URL: https://maps.google.com/?cid=4166022839385472754
===================PLACE===================
Place ID: ChIJ2QHakAijj4AR5drwpOWac9M
Name: Sigona's Signature Collections
Address: 2345 Middlefield Rd, Redwood City, CA 94063, USA
Business Status: CLOSED_PERMANENTLY
Permanently Closed: True
URL: https://maps.google.com/?cid=15236692275429956325
===================PLACE===================
Place ID: ChIJN_ReNjN2j4AR11CThpjoosc
Name: Balsam Hill
Address: 1561 Adrian Rd, Burlingame, CA 94010, USA
Business Status: CLOSED_TEMPORARILY
Permanently Closed: True
URL: https://maps.google.com/?cid=14385315901565259991
===================PLACE===================
Place ID: ChIJrclrJ1Oij4ARGae3NgmRTvo
Name: Balsam Brands
Address: 700 Jefferson Ave, Redwood City, CA 94063, USA

### 3. Determine Operational Status - Individual Search

Enter (or paste) name or address into search box. Wait for the details to be displayed.

Enter "q" to quit cell.

In [9]:
# User-specified fields (see full list here: https://developers.google.com/maps/documentation/places/web-service/place-data-fields)
fields = ['place_id', 'name', 'formatted_address', 'business_status','permanently_closed','type', 'url']

In [10]:
# Construct GooglePlaces object
api = GooglePlaces(api_key)

In [11]:
while True:
    user_input = input("Enter name or address:")
    if user_input == "q":
        break
    place = api.search_by_input(location=user_input, fields=fields)
    # Get details of the place and then print it out.
    details = api.get_place_details(place['place_id'], fields)
    api.display_fields(details, fields)
    print("OPERATIONAL STATUS:", api.get_bus_status('ChIJRxkWkd94hYARvUk7nHmfIHQ'))
    print("===========================================")
    print()

Enter name or address:DIGICOM
===================PLACE===================
Place Id: ChIJRxkWkd94hYARvUk7nHmfIHQ
Name: Digicom Electronics
Formatted Address: 7799 Pardee Ln, Oakland, CA 94621, USA
Business Status: OPERATIONAL
Permanently_Closed: N/A
Type: N/A
Url: https://maps.google.com/?cid=8367863452315371965
OPERATIONAL STATUS: Active

Enter name or address:SUISHA HOUSE
===================PLACE===================
Place Id: ChIJs9V2yKyjj4ARQfDHUHDm7j8
Name: Suisha House
Formatted Address: 2053 Broadway, Redwood City, CA 94063, USA
Business Status: CLOSED_PERMANENTLY
Permanently Closed: True
Type: N/A
Url: https://maps.google.com/?cid=4606872838912602177
OPERATIONAL STATUS: Active

Enter name or address:SEQUOIA DELI
===================PLACE===================
Place Id: ChIJ21LHlteWj4ARDgZqMtwl8ik
Name: Sequoia Premium Foods
Formatted Address: 26251 Research Pl, Hayward, CA 94545, USA
Business Status: OPERATIONAL
Permanently_Closed: N/A
Type: N/A
Url: https://maps.google.com/?cid=30225

### --------------------------------- END OF NOTEBOOK --------------------------------

### **Resources**<br>

*Readings*<br>

https://www.octoparse.com/blog/10-myths-about-web-scraping#<br>
https://medium.com/analytics-vidhya/scrapy-vs-selenium-vs-beautiful-soup-for-web-scraping-24008b6c87b8<br>


*How-To's*<br>

https://medium.com/swlh/scraping-google-maps-using-selenium-3cec08eb6a92<br>
https://python.gotrained.com/google-places-api-extracting-location-data-reviews/<br>

*Google Places API* <br>https://developers.google.com/maps/documentation/places/web-service/details#fields <br>

*Orbis PPP Master File* <br> https://drive.google.com/drive/u/0/folders/1fGrMqsLZO12pAc0MwQybE1_fWBVuA_Er